<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Redes Neuronales Artificiales, 2024 - 2025 </b></div>

# **Práctica MNIST: Data Augmentation**

In [1]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

print("Torch version: ", torch. __version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.4.1+cu121
Device:  cuda


Se fijan semillas para asegurar que el experimento sea reproducible

In [2]:
seed = 42
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
train_transform = transforms.Compose(
                    [
                    transforms.RandomRotation(3),
                    transforms.RandomAffine(degrees = 2, translate = (0.002, 0.001), scale = (0.001, 1.64)),
                    transforms.ToTensor(),
                    ])

test_transform = transforms.Compose(
                    [
                    transforms.ToTensor(),
                    ])

In [4]:
class MNIST_dataset(Dataset):

    def __init__(self, partition = "train", transform=None):

        print("\nLoading MNIST ", partition, " Dataset...")
        self.partition = partition
        self.transform = transform
        if self.partition == "train":
            self.data = torchvision.datasets.MNIST('.data/', train=True, download=True)
        else:
            self.data = torchvision.datasets.MNIST('.data/', train=False, download=True)
        print("\tTotal Len.: ", len(self.data), "\n", 50*"-")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        # Image
        image = self.data[idx][0]
        image = self.transform(image)
        # care! net expect a 784 size vector and our dataset 
        # provide 1x28x28 (channels, height, width) -> Reshape!
        image = image.view(-1)

        # Label
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()

        return {"idx": idx, "img": image, "label": label}

train_dataset = MNIST_dataset(partition="train", transform=train_transform)
test_dataset = MNIST_dataset(partition="test", transform=test_transform)


Loading MNIST  train  Dataset...
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 17519534.00it/s]


Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 470356.81it/s]


Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4369350.76it/s]


Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5272772.98it/s]

Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw

	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------


In [5]:
batch_size = 100
num_workers = multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)

Num workers 3


In [8]:
class Net(nn.Module):
    def __init__(self, sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, 10]], criterion=None):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()

        for i in range(len(sizes)-1):
            dims = sizes[i]
            self.layers.append(nn.Linear(dims[0], dims[1]))
            self.layers.append(nn.BatchNorm1d(dims[1]))
            self.layers.append(nn.ReLU())

        dims = sizes[-1]
        self.classifier = nn.Linear(dims[0], dims[1])

        self.criterion = criterion

    def forward(self, x, y=None):
        for layer in self.layers:
            x = layer(x)
        x = self.classifier(x)

        if y != None:
            loss = self.criterion(x, y)
            return loss, x
        return x

In [10]:
criterion = nn.CrossEntropyLoss()

# Instantiating the network and printing its architecture
num_classes = 10
net = Net(sizes=[
                [784, 1024], 
                [1024, 1024], 
                [1024, 1024], 
                [1024, num_classes]
                ], 
          criterion=criterion)
print(net)

Net(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)


In [11]:
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 35], gamma=0.1)
net = net.to(device)

epochs = 45

In [12]:
print("\n---- Start Training ----")
best_accuracy = -1
best_epoch = 0
for epoch in range(epochs):


    # TRAIN NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:

          images = batch["img"].to(device)
          labels = batch["label"].to(device)
          ids = batch["idx"].to('cpu').numpy()

          # zero the parameter gradients
          optimizer.zero_grad()

          #  Forward
          loss, outputs = net(images, labels)

          loss.backward()

          optimizer.step()

          # one hot -> labels
          labels = torch.argmax(labels, dim=1)
          pred = torch.argmax(outputs, dim=1)

          train_correct += pred.eq(labels).sum().item()

          # print statistics
          train_loss += loss.item()
    
        scheduler.step()
        print("\tLR: ", optimizer.param_groups[0]['lr'])

    train_loss /= len(train_dataloader.dataset)

    # TEST NETWORK
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
      with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
          for batch in tepoch:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            ids = batch["idx"].to('cpu').numpy()

            #  Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)

    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
        epoch + 1, train_loss, test_loss, 100. * train_correct / len(train_dataloader.dataset), test_accuracy
    ))

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch

        # Save best weights
        torch.save(net.state_dict(), "best_model_mnist_da.pt")
    
print("\nBEST TEST ACCURACY: ", best_accuracy, " in epoch ", best_epoch)



---- Start Training ----


Epoch 0: 100%|██████████| 600/600 [00:12<00:00, 49.08batch/s]

	LR:  0.01



Test 0: 100%|██████████| 100/100 [00:01<00:00, 77.81batch/s]

[Epoch 1] Train Loss: 0.006786 - Test Loss: 0.001429 - Train Accuracy: 77.14% - Test Accuracy: 95.51%


...

[Epoch 44] Train Loss: 0.003066 - Test Loss: 0.000337 - Train Accuracy: 89.11% - Test Accuracy: 98.86%


Epoch 44: 100%|██████████| 600/600 [00:12<00:00, 49.58batch/s]

	LR:  0.0001



Test 44: 100%|██████████| 100/100 [00:01<00:00, 82.35batch/s]

[Epoch 45] Train Loss: 0.003030 - Test Loss: 0.000340 - Train Accuracy: 89.24% - Test Accuracy: 98.80%

BEST TEST ACCURACY:  98.86  in epoch  37


In [13]:
net.load_state_dict(torch.load("best_model_mnist_da.pt"))

test_loss, test_correct = 0, 0
net.eval()
with torch.no_grad():
    with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            ids = batch["idx"].to('cpu').numpy()

            #  Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)
print("Final best acc: ", test_accuracy)

<ipython-input-13-69b22adf259e>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("best_model_mnist_da.pt"))
Test 44: 100%|██████████| 100/100 

Final best acc:  98.86
